<a href="https://colab.research.google.com/github/MerMer12345/ChickenAPI/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Mar 17 13:43:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P0             28W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!python -m pip install --upgrade pip

In [3]:
!pip install -q \
  autodistill \
  autodistill-grounded-sam \
  autodistill-yolov8 \
  roboflow \
  supervision==0.24.0

In [4]:
import os
HOME = os.getcwd()
print(HOME)
!mkdir {HOME}/images

/content
mkdir: cannot create directory ‘/content/images’: File exists


In [5]:


!mkdir {HOME}/videos
%cd {HOME}/videos

# download zip file containing videos
!wget https://media.roboflow.com/milk.zip

# unzip videos
!unzip milk.zip



mkdir: cannot create directory ‘/content/videos’: File exists
/content/videos
--2025-03-17 13:43:57--  https://media.roboflow.com/milk.zip
Resolving media.roboflow.com (media.roboflow.com)... 34.110.133.209
Connecting to media.roboflow.com (media.roboflow.com)|34.110.133.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1192719976 (1.1G) [application/zip]
Saving to: ‘milk.zip.1’

milk.zip.1          100%[===================>]   1.11G  30.9MB/s    in 42s     

2025-03-17 13:44:39 (27.4 MB/s) - ‘milk.zip.1’ saved [1192719976/1192719976]

Archive:  milk.zip
replace milk-video-1.mov? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: milk-video-1.mov        
replace milk-video-2.mov? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace milk-video-2.mov? [y]es, [n]o, [A]ll, [N]one, [r]ename: ya
  inflating: milk-video-2.mov        
replace milk-video-3.mov? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y A
  inflating: milk-video-3.mov        
r

In [6]:
VIDEO_DIR_PATH = f"{HOME}/videos"
IMAGE_DIR_PATH = f"{HOME}/images"
FRAME_STRIDE = 10

In [7]:
import supervision as sv
from tqdm.notebook import tqdm

video_paths = sv.list_files_with_extensions(
    directory=VIDEO_DIR_PATH,
    extensions=["mov", "mp4"])
print(video_paths)
TEST_VIDEO_PATHS, TRAIN_VIDEO_PATHS = video_paths[:2], video_paths[2:]

for video_path in tqdm(TRAIN_VIDEO_PATHS):
    print(f"Saving images to {IMAGE_DIR_PATH}")
    video_name = video_path.stem
    image_name_pattern = video_name + "-{:05d}.png"
    print(f"Pattern: {image_name_pattern}")
    with sv.ImageSink(target_dir_path=IMAGE_DIR_PATH, image_name_pattern=image_name_pattern) as sink:
        for image in sv.get_video_frames_generator(source_path=str(video_path), stride=FRAME_STRIDE):
            sink.save_image(image=image)



[PosixPath('/content/videos/milk-video-4.mov'), PosixPath('/content/videos/milk-video-5.mov'), PosixPath('/content/videos/milk-video-3.mov'), PosixPath('/content/videos/milk-video-8.mov'), PosixPath('/content/videos/milk-video-7.mov'), PosixPath('/content/videos/milk-video-6.mov'), PosixPath('/content/videos/milk-video-2.mov'), PosixPath('/content/videos/milk-video-1.mov')]


  0%|          | 0/6 [00:00<?, ?it/s]

Saving images to /content/images
Pattern: milk-video-3-{:05d}.png


KeyboardInterrupt: 

In [ ]:
import supervision as sv

image_paths = sv.list_files_with_extensions(
    directory=IMAGE_DIR_PATH,
    extensions=["png", "jpg", "jpg"])

print('image count:', len(image_paths))



In [ ]:


IMAGE_DIR_PATH = f"{HOME}/images"
SAMPLE_SIZE = 16
SAMPLE_GRID_SIZE = (4, 4)
SAMPLE_PLOT_SIZE = (16, 10)


import cv2
import supervision as sv

titles = [
    image_path.stem
    for image_path
    in image_paths[:SAMPLE_SIZE]]
images = [
    cv2.imread(str(image_path))
    for image_path
    in image_paths[:SAMPLE_SIZE]]

sv.plot_images_grid(images=images, titles=titles, grid_size=SAMPLE_GRID_SIZE, size=SAMPLE_PLOT_SIZE)



In [ ]:
from autodistill.detection import CaptionOntology

ontology=CaptionOntology({
    "milk bottle": "bottle",
    "blue cap": "cap"
})
DATASET_DIR_PATH = f"{HOME}/dataset"

In [ ]:


from autodistill_grounded_sam import GroundedSAM

base_model = GroundedSAM(ontology=ontology)
dataset = base_model.label(
    input_folder=IMAGE_DIR_PATH,
    extension=".png",
    output_folder=DATASET_DIR_PATH)



In [ ]:


ANNOTATIONS_DIRECTORY_PATH = f"{HOME}/dataset/train/labels"
IMAGES_DIRECTORY_PATH = f"{HOME}/dataset/train/images"
DATA_YAML_PATH = f"{HOME}/dataset/data.yaml"



In [ ]:
import supervision as sv

dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=IMAGES_DIRECTORY_PATH,
    annotations_directory_path=ANNOTATIONS_DIRECTORY_PATH,
    data_yaml_path=DATA_YAML_PATH)

len(dataset)

In [ ]:
import supervision as sv
from pathlib import Path


mask_annotator = sv.MaskAnnotator()
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

images = []
image_names = []
for i, (image_path, image, annotation) in enumerate(dataset):
    if i == SAMPLE_SIZE:
        break
    annotated_image = image.copy()
    annotated_image = mask_annotator.annotate(
        scene=annotated_image, detections=annotation)
    annotated_image = box_annotator.annotate(
        scene=annotated_image, detections=annotation)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=annotation)

    image_names.append(Path(image_path).name)
    images.append(annotated_image)

sv.plot_images_grid(
    images=images,
    titles=image_names,
    grid_size=SAMPLE_GRID_SIZE,
    size=SAMPLE_PLOT_SIZE)

In [ ]:
!locale